In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
import csv
import sqlite3
import time

In [ ]:
# helper functions
# compute hash of given protein
# compute hashes of multi fasta file
# return which proteins appear in database

### simple -icity calculation: given single protein, find bait neighbourhoods of all related proteins within the 5kb window

In [8]:
get_permissive_rep("5b58073dd6fcc6383a3b")

'c2b5fea0b52ea0ac1563'

In [33]:
get_related_baits("c2b5fea0b52ea0ac1563")

KeyboardInterrupt: 

In [28]:
def get_list_ids_fromcursor(fetchall):
    return [fetchone[0] for fetchone in fetchall]

In [35]:
def get_permissive_rep(bait_pid):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    cmd_s = "SELECT reppid FROM stringent WHERE pid = '%s'" % (bait_pid)
    cursor.execute(cmd_s)
    stringent_rep = cursor.fetchone()[0]
    cmd_p = "SELECT reppid FROM permissive WHERE pid = '%s'" % (stringent_rep)
    cursor.execute(cmd_p)
    perm_rep = cursor.fetchone()[0]
    conn.close()
    return(perm_rep)
    # cmd_p = "SELECT * FROM neighbours"
    # cursor.execute(cmd_p)
    # rv=cursor.fetchall()
    # conn.close()
    # return(rv)

def get_related_baits(perm_rep):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    cmd_get_strin_rep = "SELECT pid FROM permissive WHERE reppid = '%s'" % (perm_rep)
    cursor.execute(cmd_get_strin_rep)
    strin_reps = get_list_ids_fromcursor(cursor.fetchall())
    p100s_ls = []
    for strin_rep in strin_reps:
        cmd_get_p100 = "SELECT pid FROM stringent WHERE reppid = '%s'" % (strin_rep)
        cursor.execute(cmd_get_p100)
        p100s = get_list_ids_fromcursor(cursor.fetchall())
        p100s_ls += p100s    
    conn.close()
    return(p100s_ls)

In [60]:
def get_bait_neighbourhood(in_pid):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    #cursor.execute("DROP TABLE neighbours")
                   
    cmd_createtemptable = '''CREATE TABLE neighbours (pid, baitid, PRIMARY KEY(pid))'''
    #cursor.execute(cmd_createtemptable)
    
    cmd_getbaitneighbs = "INSERT INTO neighbours SELECT p2hash,p1hash FROM prot2protwindow WHERE p1hash = '%s'" % (in_pid)
    #print(cmd_getbaitneighbs)
    #cursor.execute(cmd_getbaitneighbs)
    cmd_getbaitneighbs = "INSERT INTO neighbours SELECT p1hash,p2hash FROM prot2protwindow WHERE p2hash = '%s'" % (in_pid)
    #cursor.execute(cmd_getbaitneighbs)
    cursor.execute("SELECT * FROM neighbours")
    rv = get_list_ids_fromcursor(cursor.fetchall())
    conn.commit()
    conn.close()
    return rv

neigh_ids = get_bait_neighbourhood("5088749308272be178d0")

    
def get_p90s(p30_id):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    cmd = "SELECT pid FROM permissive WHERE reppid = '%s'" % (p30_id)
    cursor.execute(cmd)
    p90s = get_list_ids_fromcursor(cursor.fetchall())
    conn.close()
    return(p90s)

def get_p100s(p90_id):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    cmd = "SELECT pid FROM stringent WHERE reppid = '%s'" % (p90_id)
    cursor.execute(cmd)
    p100s = get_list_ids_fromcursor(cursor.fetchall())
    conn.close()
    return(p100s)

def p100isneighbour(p100_id):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    cmd = "SELECT * FROM neighbours WHERE pid = '%s'" % (p100_id)
    cursor.execute(cmd)
    rv = len(get_list_ids_fromcursor(cursor.fetchall())) > 1
    conn.close()
    return(rv) 

def calc_icity(neighb_id):
    p30=get_permissive_rep(neighb_id)
    p90s = get_p90s(p30)
    print(p90s)
    hits = 0
    for p90 in p90s:
        p100s = get_p100s(p90)
        print(p100s)
        for p100 in p100s:
            if p100isneighbour(p100):
                hits += 1
                break
    return hits / len(p90)

INSERT INTO neighbours SELECT p2hash,p1hash FROM prot2protwindow WHERE p1hash = '5088749308272be178d0'


In [61]:
calc_icity(neigh_ids[0])

['5ec65be4a5c77e325af4', 'e633f4af36532bd61a77', 'bffb6454162dc0ff46b7', 'dc1e6a072f43855ab7ef', '915772758c2338905c98', '958997fe8686f3043a3b', '04c25bd8e8781cc1957a', 'f5373abaa62d19192e59', '08c77f1c274c28223c5d', '04287b36cbec057761ad', '883eb4bd43bc00145e3c', 'ef9f10c3a08e71984a92', '54cffe7cf1fca74ba3a1', '9d19f7789a6961384e64', 'cfe2465126f207458110', 'a6f41b0a6926a9e802e7', '0b9771a224ea8328c8e8']
['5ec65be4a5c77e325af4']
['e633f4af36532bd61a77']
['bffb6454162dc0ff46b7']
['dc1e6a072f43855ab7ef']
['915772758c2338905c98']
['958997fe8686f3043a3b']
['04c25bd8e8781cc1957a']
['f5373abaa62d19192e59']
['08c77f1c274c28223c5d']
['04287b36cbec057761ad', '0037b90d5d144f05527d']
['883eb4bd43bc00145e3c']
['ef9f10c3a08e71984a92']
['54cffe7cf1fca74ba3a1']
['9d19f7789a6961384e64']
['cfe2465126f207458110']
['a6f41b0a6926a9e802e7']
['0b9771a224ea8328c8e8']


0.0

In [58]:
t1, t2 = 0,0
for i in range(10):
    for j in range(20):
        if j>5:
            t2 += 1
            break
        

In [59]:
t2

10

In [38]:
neigh_id

['ef9f10c3a08e71984a92',
 'b0cd7fc41ed2e142a84a',
 '7426e51f0050136f0722',
 'e1912a410e1beb69fe78',
 'ce3ef3eefef3823f5aff',
 'b078cbc8fc25b6bf5574']